In [ ]:
!pip install tensorflow==1.15
import tensorflow as tf

import numpy as np

# MOUNTING THE DRIVE

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# CLONE TF/MODELS
And perform related operations

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

%cd /content/models/research
!python setup.py build
!python setup.py install

%cd /content/models/research/slim/
!python setup.py build
!python setup.py install

%tensorflow_version 1.x
%cd /content/models/research/
import os
pwd = os.getcwd()
os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim'
!python /content/models/research/object_detection/builders/model_builder_test.py

# PREPARING DIRECTORY STRUCTURE

In [ ]:
%cd /content/models/
os.mkdir('workspace')
%cd /content/models/workspace/
os.mkdir('card_detect')
%cd /content/models/workspace/card_detect/
os.mkdir('annotations')
os.mkdir('pre-trained-model')
os.mkdir('training')
!mkdir images

# CLONING THE OBJECT DETECTION REPO FROM GITHUB

In [ ]:
%cd /content/
!rm -rf obj_detect_tut
%cd /content/
!git clone https://github.com/thebeyonder001/obj_detect_tut.git

## IMPORTING IMAGES FROM THE REPO(THE DEFAULT ONES)


In [ ]:
%cd /content/models/workspace/card_detect/
os.mkdir('images')

### IMPORTING IMAGES FROM THE REPO(THE DEFAULT ONES)
from distutils.dir_util import copy_tree

fromDirectory = "/content/obj_detect_tut/images"
toDirectory = "/content/models/workspace/card_detect/images"

copy_tree(fromDirectory, toDirectory)

### GENERATING THE TFRECORDS FOR THE IMAGE DATA

In [ ]:
%tensorflow_version 1.x
%cd /content/models/research/object_detection/
!python generate_tfrecord.py --csv_input=/content/models/workspace/card_detect/images/train_labels.csv --image_dir=/content/models/workspace/card_detect/images/train --output_path=/content/models/workspace/card_detect/annotations/train.record
!python generate_tfrecord.py --csv_input=/content/models/workspace/card_detect/images/test_labels.csv --image_dir=/content/models/workspace/card_detect/images/test --output_path=/content/models/workspace/card_detect/annotations/test.record

## IMPORTING IMAGE DATA FROM DRIVE

In [ ]:
%cd /content/models/workspace/card_detect/images/
!mkdir train
!mkdir test

In [ ]:
from distutils.dir_util import copy_tree

# copy subdirectory example
fromDirectory = "/content/drive/My Drive/GWC_imgs/dataset/test"
toDirectory = "/content/models/workspace/card_detect/images/test"

copy_tree(fromDirectory, toDirectory)

In [ ]:
from distutils.dir_util import copy_tree

# copy subdirectory example
fromDirectory = "/content/drive/My Drive/GWC_imgs/dataset/train"
toDirectory = "/content/models/workspace/card_detect/images/train"

copy_tree(fromDirectory, toDirectory)

### LOADING TFRECORDS FROM DRIVE

In [ ]:
import shutil

original = r'/content/drive/My Drive/GWC_imgs/dataset/test.record'
target = r'/content/models/workspace/card_detect/annotations/test.record'

shutil.copyfile(original, target)

In [ ]:
original = r'/content/drive/My Drive/GWC_imgs/dataset/train.record'
target = r'/content/models/workspace/card_detect/annotations/train.record'

shutil.copyfile(original, target)

In [ ]:
from distutils.dir_util import copy_tree

# copy subdirectory example
fromDirectory = "/content/obj_detect_tut/training"
toDirectory = "/content/models/workspace/card_detect/training"

copy_tree(fromDirectory, toDirectory)

# DOWNLOADING THE SPECIFIC PRE-TRAINED MODEL

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = 'faster_rcnn_inception_v2_coco_2018_01_28' + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/workspace/card_detect/pre-trained-model/'

if not (os.path.exists(MODEL_FILE)):
    #urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE,DEST_DIR+MODEL_FILE)

# opener = urllib.request.URLopener()
# filename,_ = opener.retrieve(url, 'Test.pdf')

%cd /content/models/workspace/card_detect/pre-trained-model/
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
#if (os.path.exists(DEST_DIR)):
#    shutil.rmtree(DEST_DIR)
#os.rename(MODEL, DEST_DIR)

In [ ]:
from distutils.dir_util import copy_tree
import distutils
# copy subdirectory example
fromDirectory = "/content/models/workspace/card_detect/pre-trained-model/"
toDirectory = "/content/models/workspace/card_detect/"

#copy_tree(fromDirectory, toDirectory)
distutils.dir_util._path_created = {}
distutils.dir_util.copy_tree(fromDirectory,toDirectory)

# TRAINING

## TRAINING

In [ ]:
# src = "/content/obj_detect_tut/training/faster_rcnn_inception_v2_pets.config"
# des = "/content/models/workspace/card_detect/training/faster_rcnn_inception_v2_pets.config"
# shutil.copyfile(src,des)

# src = "/content/obj_detect_tut/training/labelmap.pbtxt"
# des = "/content/models/workspace/card_detect/training/labelmap.pbtxt"
# shutil.copyfile(src,des)

original = r'/content/models/research/object_detection/model_main.py'
target = r'/content/models/workspace/card_detect/model_main.py'

shutil.copyfile(original, target)

In [ ]:
%cd /content/models/workspace/card_detect/
!python model_main.py --alsologtostderr --model_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_pets.config

## EXPORTING INFERENCE GRAPH

In [ ]:
%cd /content/models/workspace/card_detect/
!rm -rf inference_graph

In [ ]:

import shutil

original = r'/content/models/research/object_detection/export_inference_graph.py'
target = r'/content/models/workspace/card_detect/export_inference_graph.py'

shutil.copyfile(original, target)

In [ ]:
%cd /content/models/workspace/card_detect
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-28383 --output_directory inference_graph


# TESTING

In [ ]:
import os
#os.listdir("/content/models/workspace/card_detect/images/test")

In [ ]:
import shutil

original = r'/content/drive/My Drive/GWC_imgs/dataset/test/d63350a3d.jpg'
target = r'/content/models/workspace/card_detect/test.jpg'

shutil.copyfile(original, target)

In [ ]:
%cd /content/models/workspace/card_detect/
from google.colab.patches import cv2_imshow
import cv2
a=cv2.imread("test.jpg")
a = cv2.resize(a,(512,512))
cv2_imshow(a)

In [ ]:
def gwc_test(path,img):
  %cd /content/models/research/object_detection/
  ######## Image Object Detection Using Tensorflow-trained Classifier #########
  #
  # Author: Evan Juras
  # Date: 1/15/18
  # Description: 
  # This program uses a TensorFlow-trained neural network to perform object detection.
  # It loads the classifier and uses it to perform object detection on an image.
  # It draws boxes, scores, and labels around the objects of interest in the image.

  ## Some of the code is copied from Google's example at
  ## https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

  ## and some is copied from Dat Tran's example at
  ## https://github.com/datitran/object_detector_app/blob/master/object_detection_app.py

  ## but I changed it to make it more understandable to me.
  import sys

  # This is needed since the notebook is stored in the object_detection folder.
  sys.path.append("..")

  # Import utilites
  from utils import label_map_util
  from utils import visualization_utils as vis_util

  # Name of the directory containing the object detection module we're using
  MODEL_NAME = 'inference_graph'
  #IMAGE_NAME = 'test.jpg'
  IMAGE_NAME = img
  # Grab path to current working directory
  #CWD_PATH = os.getcwd()
  CWD_PATH = "/content/models/workspace/card_detect/"
  # Path to frozen detection graph .pb file, which contains the model that is used
  # for object detection.
  PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

  # Path to label map file
  PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

  # Path to image
  PATH_TO_IMAGE = path+img#os.path.join(CWD_PATH,IMAGE_NAME)

  # Number of classes the object detector can identify
  NUM_CLASSES = 6

  # Load the label map.
  # Label maps map indices to category names, so that when our convolution
  # network predicts `5`, we know that this corresponds to `king`.
  # Here we use internal utility functions, but anything that returns a
  # dictionary mapping integers to appropriate string labels would be fine
  label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
  categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
  category_index = label_map_util.create_category_index(categories)

  # Load the Tensorflow model into memory.
  detection_graph = tf.Graph()
  with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
          serialized_graph = fid.read()
          od_graph_def.ParseFromString(serialized_graph)
          tf.import_graph_def(od_graph_def, name='')

      sess = tf.Session(graph=detection_graph)

  # Define input and output tensors (i.e. data) for the object detection classifier

  # Input tensor is the image
  image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

  # Output tensors are the detection boxes, scores, and classes
  # Each box represents a part of the image where a particular object was detected
  detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
  
  # Each score represents level of confidence for each of the objects.
  # The score is shown on the result image, together with the class label.
  detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
  detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

  # Number of objects detected
  num_detections = detection_graph.get_tensor_by_name('num_detections:0')

  # Load image using OpenCV and
  # expand image dimensions to have shape: [1, None, None, 3]
  # i.e. a single-column array, where each item in the column has the pixel RGB value
  image = cv2.imread(PATH_TO_IMAGE)
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image_expanded = np.expand_dims(image_rgb, axis=0)

  # Perform the actual detection by running the model with the image as input
  (boxes, scores, classes, num) = sess.run(
      [detection_boxes, detection_scores, detection_classes, num_detections],
      feed_dict={image_tensor: image_expanded})
  # Draw the results of the detection (aka 'visulaize the results')

  vis_util.visualize_boxes_and_labels_on_image_array(
      image,
      np.squeeze(boxes),
      np.squeeze(classes).astype(np.int32),
      np.squeeze(scores),
      category_index,
      use_normalized_coordinates=True,
      line_thickness=8,
      min_score_thresh=0.50)

  # All the results have been drawn on image. Now display the image.
  #image = cv2.resize(image,(512,512))

  cv2_imshow(image)

  # Press any key to close the image
  cv2.waitKey(0)
  # sess = tf.Session()
  # with sess.as_default():
  #     print(detection_boxes.eval())
  #a = detection_boxes.numpy()
  # Clean up
  cv2.destroyAllWindows()
  return boxes,scores

In [ ]:
path = "/content/models/workspace/card_detect/"
img = "test.jpg"
b,s = gwc_test(path,img)

In [ ]:
path = "/content/models/workspace/card_detect/"
img = "test.jpg"
t,n = gwc_test(path,img)

## STORING THE TRAINED MODEL FILES IN DRIVE

In [ ]:
# step = 24171
# src = "/content/models/workspace/card_detect/training/"
# des = "/content/drive/My Drive/GWC_imgs/"
# m = "model.ckpt-"
# i = [".data-00000-of-00001",".index",".meta"]
# # shutil.copyfile(src+"checkpoint",des+"checkpint")
# # shutil.copyfile(src+"graph.pbtxt",des+"graph.pbtxt")
# for ex in i:
#   shutil.copyfile(src+m+str(step)+ex,des+m+str(step)+ex)

In [ ]:
src = "/content/models/workspace/card_detect/inference_graph/"
des = "/content/drive/My Drive/Trained_models/GWC/"
from distutils import dir_util
dir_util.copy_tree(src,des)

In [ ]:
import tensorflow as tf
%tensorflow_version 1.x

In [ ]:
import tensorflow.compat.v1 as tf
import numpy as np

tf.disable_v2_behavior()

In [ ]:
!pip install tensorflow==1.15

# TESTING ON KAGGLE

In [ ]:
%cd /content/
!mkdir test
from shutil import copyfile
copyfile("/content/test.zip","/content/test/test.zip")
%cd /content/test/
!unzip test.zip

In [ ]:
path = "/content/test/"
lst = []
for img in os.listdir(path):
  if img!="test.zip":
    c = 0
    box,scores = gwc_test(path,img)
    for score in scores[0]:
      d = []
      d.append(img.replace(".jpg",""))
      if score>=0.50:
        xmin = int(box[0,c,0]*1024)
        ymin = int(box[0,c,1]*1024)
        xmax = int(box[0,c,2]*1024)
        ymax = int(box[0,c,3]*1024)
        d.append(f"{score} {xmin} {ymin} {xmax} {ymax}")
        lst.append(d)
      c = c+1

In [ ]:
len(lst)

In [ ]:
import pandas as pd

In [ ]:
[]

In [ ]:
df = pd.DataFrame(lst,columns = ["image_id","PredictionString"])
df.head()

In [ ]:
print(df)

In [ ]:
image  = cv2.imread("/content/test/2fd875eaa.jpg")
#image[:,:,0] = 50
image[:,:,1] = 150
#image[:,:,2] = 200
cv2_imshow(image)

In [ ]:
image  = cv2.imread("/content/test/2fd875eaa.jpg")
#image[:,:,0] = 50
image[:,:,1] = 150
image[:,:,2] = 50
cv2_imshow(image)

In [ ]:
image  = cv2.imread("/content/test/2fd875eaa.jpg")
#image[:,:,0] = 50
image[:,:,2] = 120
image[:,:,1] = 225
cv2_imshow(image)